## LQTMOMENT Tutorial 3: Calculating Moment Magnitudes from a Batch of Earthquakes

> **⚠️ CAUTION ⚠️**
> 
> Before continuing this tutorial, make sure you have already seen the 1st and 2nd LQT tutorials.

### 1. Programmatic Approach

#### A. Import Moment Magnitude Estimator Module

For calculating a batch of earthquakes data we need to import `magnitude_estimator` function from `lqtmoment` package.

In [11]:
from lqtmoment import magnitude_estimator
from pathlib import Path

### Initialize Input/Output File/Dir

In [16]:
# Initialize directories object
dirs = {
    "hypo_dir": r"D:\lqt-moment-magnitude\tests\sample_tests_data\data\catalog\hypo_catalog.xlsx",
    "pick_dir": r"D:\lqt-moment-magnitude\tests\sample_tests_data\data\catalog\picking_catalog.xlsx",
    "station_dir": r"D:\lqt-moment-magnitude\tests\sample_tests_data\data\station\station.xlsx",
    "output_dir": r"D:\lqt-moment-magnitude\tests\sample_tests_data\results\lqt_catalog"
}

